In [1]:
import logging
import asyncio
import os
import sys
import json

import pandas as pd
import matplotlib.pyplot as plt

# XXX temporary hack to import from src
try:
    path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), "src")
except:
    path = "~/gith/domschl/indrajala/python_indrajala/indralib/src"
    # expand ~
    path = os.path.expanduser(path)
print(path)
sys.path.append(path)

from indra_event import IndraEvent
from indra_client import IndraClient
from indra_downloader import IndraDownloader

/Users/dsc/gith/domschl/indrajala/python_indrajala/indralib/src


In [2]:
logging.basicConfig(
    format="%(asctime)s %(levelname)s %(name)s %(message)s", level=logging.INFO
)


In [3]:

dl = IndraDownloader(cache_dir="geodata/cache")
dfs = dl.get_datasets(data_sources_dir="geodata/data_sources")
for df_name in dfs:
    print("-----------------------------------------------")
    print(df_name)
    print(dfs[df_name]["metadata"])
    print(dfs[df_name]["data"].head())
print(f"Number of datasets: {len(dfs)}")


2023-06-27 10:21:55,501 INFO Downloader processing: geodata/data_sources/MaunaLoaCO2MonthlyMean.toml
2023-06-27 10:21:56,301 INFO Downloader Read https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv from cache at geodata/cache/co2_mm_mlo.csv
2023-06-27 10:21:56,302 INFO Downloader Creating dataset manualoaco2monthlymean
2023-06-27 10:21:56,309 ERROR Downloader Failed to apply transform: invalid syntax (<string>, line 1)
2023-06-27 10:21:56,312 ERROR Downloader Data is not a dataframe, cannot apply transform
2023-06-27 10:21:56,312 ERROR Downloader Data is not a dataframe, cannot apply transform
2023-06-27 10:21:56,312 INFO Downloader Indra import: time_jd_start, mean/co NOT IMPLEMENTED YET
2023-06-27 10:21:56,313 INFO Downloader Indra import: time_jd_start, deseasonalized/co2 NOT IMPLEMENTED YET
2023-06-27 10:21:56,314 INFO Downloader processing: geodata/data_sources/EuropeanMeanTemperatureSinceRomanTime_EuroMed2k.toml


   year  month  decimal date  average  deseasonalized  ndays  sdev  unc
0  1958      3     1958.2027   315.70          314.43    NaN   NaN  NaN
1  1958      4     1958.2877   317.45          315.16    NaN   NaN  NaN
2  1958      5     1958.3699   317.51          314.71    NaN   NaN  NaN
3  1958      6     1958.4548   317.24          315.14    NaN   NaN  NaN
4  1958      7     1958.5370   315.86          315.18    NaN   NaN  NaN


2023-06-27 10:21:57,125 INFO Downloader Read https://www.ncei.noaa.gov/pub/data/paleo/pages2k/EuroMed2k/eujja_2krecon_nested_cps.txt from cache at geodata/cache/eujja_2krecon_nested_cps.txt
2023-06-27 10:21:57,126 INFO Downloader Creating dataset euromed2k
2023-06-27 10:21:57,131 INFO Downloader processing: geodata/data_sources/LawDome2006.toml
2023-06-27 10:21:58,164 INFO Downloader Read https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/law/law2006.txt from cache at geodata/cache/law2006.txt
2023-06-27 10:21:58,164 INFO Downloader Creating dataset LawDome2006_CO2_CH4_N20
2023-06-27 10:21:58,178 INFO Downloader Creating dataset LawDome2006_CO2_Core_DSS_Eth1996
2023-06-27 10:21:58,180 INFO Downloader Creating dataset LawDome2006_CO2_Core_DSS_MacFM20042006
2023-06-27 10:21:58,181 INFO Downloader Creating dataset LawDome2006_CO2_Core_DE08-2_Eth1996
2023-06-27 10:21:58,183 INFO Downloader Creating dataset LawDome2006_CO2_Core_DE08-2_MacFM20042006
2023-06-27 10:21:58,185 INFO Down

-----------------------------------------------
euromed2k
{'title': 'European Mean and Spatial Summer Temperature Reconstructions Since Roman Times', 'description': 'Spatial (5x5 degree grid) reconstructions of European summer (June-August) temperature back to 755 CE based on a Bayesian hierarchical modelling (BHM), plus estimates of the European mean temperature variation since 138 BCE, based on composite-plus-scaling (CPS).', 'authors': ['Luterbacher, J.', 'Werner, J.P.', 'Smerdon, J.E.', 'Fernández-Donado, L.', 'González-Rouco, J.F.', 'Barriopedro, D.', 'Ljungqvist, F.C.', 'Büntgen, U.', 'Zorita, E.', 'Wagner, S.', 'Esper, J.', 'McCarroll, D.', 'Toreti, A.', 'Frank, D.C.', 'Jungclaus, J.H.', 'Barriendos, M.', 'Bertolin, C.', 'Bothe, O.', 'Brázdil, R.', 'Camuffo, D.', 'Dobrovolný, P.', 'Gagen, M.H.', 'García-Bustamante, E.', 'Ge, Q.', 'Gómez-Navarro, J.J.', 'Guiot, J.', 'Hao, Z.', 'Hegerl, G.C.', 'Holmgren, K.', 'Klimenko, V.V.', 'Martín-Chivelet, J.', 'Pfister, C.', 'Roberts, N.', '

In [4]:
from datetime import timedelta, datetime, timezone

def convert_partial_year(number):
    year = int(number)
    d = timedelta(days=(number - year) * 365.25)
    day_one = datetime(year, 1, 1, tzinfo=timezone.utc)
    date = d + day_one
    return date

In [5]:
cl = IndraClient(config_file="ws_indra.toml", verbose = True)
await cl.init_connection(verbose=True)

2023-06-27 10:22:03,819 ERROR IndraClient Could not connect to wss://localhost:8082: Multiple exceptions: [Errno 61] Connect call failed ('127.0.0.1', 8082), [Errno 61] Connect call failed ('::1', 8082, 0, 0)


In [6]:
async def async_entire_history(domain):
    # data = await asyncio.create_task(cl.get_wait_history(domain, -1000000000))
    return await cl.get_wait_history(domain, -1000000000)

In [7]:
dataset = 'manualoaco2monthlymean'
df = dfs[dataset]['data']
print(df.head())
md = dfs[dataset]['metadata']
print(md['indra_id'], md['indra_domain'])
 
domain = md['indra_domain'].replace("{indra_id}", md['indra_id']).replace(f"{dataset}", dataset)
print(domain)

# existing_data = await async_entire_history(domain)
# print(f"Start, existing data: {len(existing_data)} records")

# Enum rows:
dups=0
for i, row in df.iterrows():
    # get column 'decimal year':
    # print(f"{i}: {row['year']}-{row['month']}  z{row['decimal date']}, {row['average']}")
    # convert fractional year into datetime:
    dt = convert_partial_year(row['decimal date'])
    jd = IndraEvent.datetime2julian(dt)
    found = False
    for (j, e) in enumerate(existing_data):
        # print(jd,e[0])
        if abs(jd-e[0])<0.0001:
            # print(f"Found existing data for {jd}")
            found = True
            dups += 1
            break
    if found:
        continue
    ie = IndraEvent()
    ie.domain = domain
    ie.from_id = "py/MaunaLoaMonthlyCO2"
    ie.to_scope = "public"
    ie.time_jd_start = jd
    ie.data_type = "number/float/co2"
    ie.data = json.dumps(row['average'])
    print("Send new: ", ie.to_json())
    # await cl.send_event(ie)
print(f"Already existing data: {dups} records, new: {len(df)-dups} records")

KeyError: 'manualoaco2monthlymean'

In [ ]:
domain

'$event/geodata/historical/{dataset}/{column}'

In [ ]:
data = await async_entire_history(domain)


2023-06-22 09:54:15,707 INFO IndraClient Sending: {"domain": "$trx/db/req/history", "from_id": "ws/python", "uuid4": "1d3b74ec-6dca-41bb-ae57-35c5c5ef9a48", "to_scope": "", "time_jd_start": 2460117.82934847, "data_type": "historyrequest", "data": "{\"domain\": \"$event/geodata/historical/{dataset}/{column}\", \"time_jd_start\": -1000000000, \"time_jd_end\": null, \"limit\": null, \"data_type\": \"number/float%\", \"mode\": \"Sample\"}", "auth_hash": "", "time_jd_end": null}
2023-06-22 09:54:15,711 INFO IndraClient Received message: {"domain":"Ws.1/127.0.0.1:50848","from_id":"SQLx.1","uuid4":"1d3b74ec-6dca-41bb-ae57-35c5c5ef9a48","to_scope":"$event/geodata/historical/{dataset}/{column}","time_jd_start":2460117.829348492,"data_type":"vector/tuple/jd/float","data":"[]","auth_hash":null,"time_jd_end":2460117.829348503}
2023-06-22 09:54:15,712 INFO IndraClient ---------------------------------------------------------------
2023-06-22 09:54:15,712 INFO IndraClient Future: trx event $event/ge

In [ ]:
data

[]

2023-06-22 09:57:08,713 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,715 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,715 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,716 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,716 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,717 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,717 ERROR IndraClient Could not receive message: no close frame received or sent
2023-06-22 09:57:08,718 ERROR IndraClient Could not receive message: no close frame received or sent


In [ ]:
df = pd.DataFrame([1,2,3],[3,4,5])